# Apprentissage sur les images

Entraine le modèle sur des images et pas sur les vidéos.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

In [5]:
train_dataset = image_dataset_from_directory(
    directory='DB/train/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224)
)

test_dataset = image_dataset_from_directory(
    directory="DB/test/",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224, 224)
)

Found 4736 files belonging to 5 classes.
Found 3568 files belonging to 5 classes.


In [10]:
from keras.applications import VGG16
from keras.applications import ResNet101V2
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

resnet = ResNet101V2(include_top=False, weights="imagenet")
resnet.trainable = False

modele = Sequential([
    Input((224, 224, 3)),
    resnet,
    Flatten(),
    Dense(1024),
    Dropout(0.1),
    Dense(1024),
    Dense(5, activation="softmax")
])

modele.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101v2 (Functional)     (None, None, None, 2048)  42626560  
_________________________________________________________________
flatten_3 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 5125      
Total params: 146,442,757
Trainable params: 103,816,197
Non-trainable params: 42,626,560
_______________________________

In [ ]:
modele.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = modele.fit(train_dataset, epochs=20, verbose=1, batch_size=32)

In [ ]:
modele.save("resnet101v2")